---
modules

In [ ]:
import pandas as pd
import numpy as np

import geopandas as gpd

from keplergl import KeplerGl
import googlemaps
import h3

---
creating datasets

bike theft data  

[Website zum Datensatz daten.berlin.de](https://daten.berlin.de/datensaetze/fahrraddiebstahl-berlin)

- Lizenz:
    - Creative Commons Namensnennung CC-BY License
- Kategorie:
   - Öffentliche Sicherheit
- Geographische Abdeckung: 
    - Berlin
- Geographische Granularität: 
    - Berlin
- Zeitliche Granularität: 
    - Tag
- Veröffentlicht: 
    - 08.09.2021
- Aktualisiert: 
    - 09.09.2021
- Veröffentlichende Stelle: 
    - Polizei Berlin LKA St 14
- E-Mail Kontakt: 
    - onlineredaktion AT polizei.berlin.de

In [ ]:
# df = pd.read_csv('data/Fahrraddiebstahl.csv') # not running, encoding must be set!
df = pd.read_csv('data/Fahrraddiebstahl.csv', encoding='latin-1')
df.columns = df.columns.str.lower()
df.head(2)

In [ ]:
df['lor_str'] = df['lor'].map(lambda x: (str(x)))
df['lor_zf'] = df['lor_str'].apply(lambda x: x.zfill(8)) #lor zero fill
df['plr_id'] = df['lor_zf'].map(lambda x: (x))
df['bzr_id'] = df['lor_zf'].map(lambda x: (x)[-6:])
df['pgr_id'] = df['lor_zf'].map(lambda x: (x)[-4:])

df.drop(columns='lor_str', inplace=True)

df.columns

LOR shapefile to geopandas dataframe

_data/LOR_SHP_2021/lor_plr.shp_  
_data/LOR_SHP_2021/lor_bzr.shp_  
_data/LOR_SHP_2021/lor_pgr.shp_  

PLR: 8 digits, most granular  
BZR: 6 digits  
PGR: 4 digits

In [ ]:
gdf = gpd.GeoDataFrame.from_file('data/LOR_SHP_2021/lor_plr.shp') #.replace({-999: None})
gdf.columns = gdf.columns.str.lower()

In [ ]:
#transforms dataframe to new crs - coordinate reference system, here utm to wgs/latlong
gdf_wgs = gdf.to_crs({'proj':'longlat', 'ellps':'WGS84', 'datum':'WGS84'})

In [ ]:
# creating centroids of polygons as gpd point wkt
gdf_wgs['points'] = gdf_wgs['geometry'].centroid

# extracting lat and long from the point geometry
gdf_wgs["lat"] = gdf_wgs.points.y
gdf_wgs["lon"] = gdf_wgs.points.x


# drop point geometry to not confuse kepler (to gdf geometry columns are misleading)
gdf_wgs.drop(columns='points', inplace=True)

In [ ]:
gdf_wgs.info()

---
plotting shapes/centroids in kepler

In [ ]:
# map_1 = KeplerGl(height=800) #create an instance of this class

In [ ]:
# map_1.add_data(data=gdf_wgs, name='LOR')

In [ ]:
# map_1

---
merging

In [ ]:
df.info()

In [ ]:
gdf_wgs.info()

In [ ]:
df_combined = gdf_wgs.merge(df, on='plr_id')
df_combined.info()

In [ ]:
df_combined.dropna(axis=0, inplace=True)
df_combined.shape

In [ ]:
type(df_combined.geometry[0])

---
plotting df_combined

39.407 observations: rendering interrupted aftzer 6 minutes  
1.000 observations: map not zoomable, loses shapes  

so ... creating a grouped dataframe counting the thefts

In [ ]:
df_plr_group = df.groupby(by='plr_id').count().reset_index()
df_plr_group = df_plr_group[['plr_id', 'delikt']]
df_plr_group.head()

In [ ]:
df_combined = gdf_wgs.merge(df_plr_group, on='plr_id')
df_combined.info()

adding this leads to whitescreen, wtf?!

In [ ]:
# adding h3 hexbin h3_id
# df_combined['hex'] = df_combined.apply(lambda x: h3.geo_to_h3(x.lat, x.lon, resolution = 7), axis = 1)

In [ ]:
df_combined.info()

In [ ]:
map_2 = KeplerGl(height=800)


In [ ]:
map_2.add_data(data=df_combined, name='PLR')

In [ ]:
map_2

In [ ]:
import h3
import googlemaps